# Autograd: 自动求导

在PyTorch里，神经网络的核心是autograd包。我们首先简单了解一下，然后训练我们的第一个神经网络。

autograd包为所有tensor的运算提供自动求导功能。它是一个运行时定义的框架，这意味着反向传播由正在执行的代码决定的，因此每次迭代可以是不同的。

我们先来看一些简单的例子。

## Tensor

torch.Tensor 是Autograd包里的核心类。如果把它的属性.requires_grad设置为true，它将开始追踪所有的运算。当完成了所有的运算后，通过调用.backward()方法，可以自动得到所有的导数。这个Tensor的所有梯度将会自动累加到.grad属性。

.detach()方法可以阻止运算历史被追踪，也可以阻止未来的运算被跟踪。

为了阻止跟踪历史和使用不必要的内存，你可以把代码块包装在with torch.no_grad():。这在评估模型时（指训练后的模型）非常有用，因为模型中仍然会有requires_grad=True的参数，但我们不需要计算他们的导数。

还有一个类对于自动求导的实现非常重要——Function。

Tensor和Function是相互联系的，他们共同构建了一个无环图（acyclic graph），这个图编码了一个完整的运算历史。

每个Tensor都有.grad_fn属性，该属性引用了创建Tensor自身的Function（除非这个Tensor是用户手动创建的，即这个Tensor的grad_fn是None）。

如果你想计算导数，你可以调用.backward()方法。如果Tensor是一个scalar（它只包含一个元素），则不需要为backward指定任何参数。但是如果它有更多元素，则需要指定一个gradiant参数，该参数是尺寸匹配的Tensor。

In [3]:
import torch

创建一个Tensor，并设置它的requires_grad=True,用以追踪计算过程。

In [4]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


对它做一个运算。

In [5]:
y =  x+ 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y 作为一个运算结果而创建，因此它具有grad_fn属性。

In [6]:
print(y.grad_fn)

再做一些运算。

In [7]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


.requires_grad_()能够改变一个tensor的.requires_grad属性。如果没有给出参数，那么.requires_grad将被初始化为False。

In [8]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
#a.requires_grad_(True)
a.requires_grad = True
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)
print(b.requires_grad)

False
True
True


## Gradients

现在让我们开始学习反向传播。因为out（上文）是一个标量，所以out.backward()等价于out.backward(torch.tensor(1.)。

In [9]:
out.backward()

输出  d(out)/dx  看看

In [10]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


我们得到了一个值为``4.5``的矩阵。令out为$o$。那么$o=\frac{1}{4}\sum_i z_i$，
$z_i = 3(x_i+2)^2$
$z_i\bigr\rvert_{x_i=1} = 27$。
因此，
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i + 2)$,
进而有，
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$。

out的例子解释了标量如何求导。
如果我们有一个向量$\vec{y}$，一个向量$\vec{x}$，有函数$\vec{y}$=f($\vec{x}$)，那么$\vec{y}$对$\vec{x}$求导的结果将表示为一个雅可比矩阵：
\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}

一般说来，torch.autograd是一个计算雅可比积的一个引擎。给定任意向量$v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$，计算$v^{T}\cdot J$。如果$v$是一个标量函数$l=g\left(\vec{y}\right)$的导数，也就是说$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$，那么根据链式法则，向量的雅可比积就是函数值$l$对自变量$\vec{x}$的导数：
\begin{align}J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

雅可比积的这一性质使得将外部梯度输入到具有非标量输出的模型中变得非常方便。

下面我们来看一个雅可比积的简单例子：

In [11]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([1584.2579, -269.8928,  609.0181], grad_fn=<MulBackward0>)


In [12]:
#y.backward()
print(x)
print(x.grad)

tensor([ 1.5471, -0.2636,  0.5947], requires_grad=True)
None


In [13]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v) #我还没有弄清楚这个v的意义，简单理解，就是给了一个方向求方向导数。

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


把代码块封装在with torch.no_grad():可以很容易地阻止pytorch跟踪.requires_grad=True的Tensor的运算历史。

In [14]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


或者，使用.detach()获得一个相同的Tensor，但是不含梯度。

In [15]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)


autograd.Function的文档在[这里]( https://pytorch.org/docs/stable/autograd.html#function)